In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai
import asyncio
import json, os, re, tempfile, time, math
from datetime import datetime
from typing import List, Dict, Any
from openai import OpenAI
from copy import deepcopy
from pathlib import Path

In [ ]:
import os


os.environ["OPENAI_API_KEY"] =
os.environ["CACHE_DIR"] = "./model_cache"


print("OPENAI_API_KEY:", os.environ.get("OPENAI_API_KEY"))
print("CACHE_DIR:", os.environ.get("CACHE_DIR"))

In [ ]:
import os
import torch
import openai
from openai import OpenAI
import abc
from transformers import AutoTokenizer, AutoModelForCausalLM
import math
from concurrent.futures import ThreadPoolExecutor

def chunkify(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

# Base abstract class for model pipelines.
class ModelPipelineBase(abc.ABC):
    @abc.abstractmethod
    def call_model(self, prompt: str) -> str:
        """Send a prompt to the model and return its response."""
        pass
    def call_model_batch(self, prompts: list[str], batch_size: int = 16) -> list[str]:
        """
        Split `prompts` into chunks of size batch_size, then call `call_model`
        on each chunk in parallel threads.
        """
        results = []
        for chunk in chunkify(prompts, batch_size):
            with ThreadPoolExecutor(max_workers=len(chunk)) as ex:
                futures = [ex.submit(self.call_model, p) for p in chunk]
                for f in futures:
                    results.append(f.result())
        return results

# Detailed Hugging Face pipeline that uses AutoTokenizer and AutoModelForCausalLM.
class HuggingFaceDetailedPipeline(ModelPipelineBase):
    def __init__(
        self,
        model_name: str,
        cache_dir: str = None,
        torch_dtype: torch.dtype = torch.float16,
        temperature: float = 0.0,
        device: int = -1,  # -1 means CPU; set to GPU device index if available.
        max_length: int = 150
    ):
        """
        Initialize the Hugging Face pipeline.
        :param model_name: Name or path of the model.
        :param cache_dir: Directory to cache the model. If not provided in argument or environment, no cache_dir is used.
        :param torch_dtype: The torch dtype for the model weights.
        :param temperature: Sampling temperature (0 for deterministic).
        :param device: Device index (-1 for CPU, >=0 for GPU).
        :param max_length: Maximum length of generated text.
        """
        self.cache_dir = cache_dir or os.environ.get("CACHE_DIR")
        self.model_name = model_name
        self.torch_dtype = torch_dtype
        self.temperature = temperature
        self.device = device
        self.max_length = max_length

        if self.cache_dir:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=self.cache_dir)
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                cache_dir=self.cache_dir,
                torch_dtype=self.torch_dtype
            )
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=self.torch_dtype
            )

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        if self.device >= 0:
            self.model.to(torch.device(f"cuda:{self.device}"))
        else:
            self.model.to(torch.device("cpu"))

    def call_model(self, prompt: str) -> str:
        """
        Generate a response from the model given the prompt.
        """
        return self._generate(prompt)

    def _generate(self, prompt: str) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt")
        # Move inputs to the appropriate device.
        inputs = {key: value.to(self.model.device) for key, value in inputs.items()}
        outputs = self.model.generate(
            **inputs,
            max_length=self.max_length,
            temperature=self.temperature,
            pad_token_id=self.tokenizer.eos_token_id,
            do_sample=self.temperature > 0  # If temperature==0, generation is deterministic.
        )
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return generated_text


class GPTAPIPipeline(ModelPipelineBase):
    def __init__(self, model: str = "gpt-3.5-turbo", temperature: float = 0.0, max_tokens: int = 150):
        """
        Initialize the API pipeline.
        Expects the OPENAI_API_KEY to be set in environment variables.
        :param model: The model name to use (e.g., 'gpt-3.5-turbo').
        :param temperature: Sampling temperature for the API (default 0.0 for deterministic output).
        :param max_tokens: Maximum tokens to generate.
        """
        self.api_key = os.environ.get("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OPENAI_API_KEY is not set in the environment variables.")
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.client = OpenAI(api_key=self.api_key)

    def call_model(self, prompt: str) -> str:
        return self._api_call(prompt)

    def _api_call(self, prompt: str) -> str:

        completion = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
            max_tokens=self.max_tokens,
        )
        return completion.choices[0].message.content

class OpenAIBatchPipeline(ModelPipelineBase):
    """
    Re-implements `call_model_batch` using the Batch API.
    `call_model` falls back to the sync endpoint so your existing single-call
    code (e.g. debugging) still works.
    """
    def __init__(self, max_tokens: int = 150, temperature: float = 0.0):
        self.max_tokens  = max_tokens
        self.temperature = temperature

    # --- single prompt (rare) ------------------------------------------------
    def call_model(self, prompt: str) -> str:
        resp = client.chat.completions.create(
            model       = OPENAI_MODEL_ID,
            messages    = [{"role": "user", "content": prompt}],
            max_tokens  = self.max_tokens,
            temperature = self.temperature
        )
        return resp.choices[0].message.content

    # --- many prompts (main path) -------------------------------------------
    def call_model_batch(self, prompts: List[str],
                         batch_size: int = 64) -> List[str]:
        """
        `batch_size` is irrelevant now; we always send the whole list through
        the Batch API, chunked automatically by `run_batch`.
        """
        lines = [_make_jsonl_line(f"req-{i}", p, self.max_tokens,
                                  self.temperature)
                 for i, p in enumerate(prompts)]
        replies = run_batch(lines)
        # Preserve original order
        return [replies[f"req-{i}"] for i in range(len(prompts))]

class GPTAPILogprobPipeline(ModelPipelineBase):
    """
    Pipeline to call OpenAI Chat API and record both the parsed answer and its log probability.
    """
    def __init__(self, model: str = "gpt-3.5-turbo", temperature: float = 0.0, max_tokens: int = 10):
        self.api_key = os.environ.get("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OPENAI_API_KEY must be set in environment variables.")
        self.client = OpenAI(api_key=self.api_key)
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens

    def call_model(self, prompt: str) -> tuple[str, float]:  # returns (answer, total_logprob)
        # Call the Chat API with logprobs
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            logprobs=True,
            top_logprobs=1  # for each token, only the chosen token's logprob
        )
        choice = resp.choices[0]
        text = choice.message.content.strip()
        # extract year or fallback
        m = re.search(r"\b(\d{4})\b", text)
        answer = m.group(1) if m else text

        # sum of generated token logprobs
        token_logps = [tok.logprob for tok in choice.logprobs.content]
        total_logp = sum(token_logps)
        return answer, total_logp


# Factory class to create the appropriate pipeline.
class ModelPipelineFactory:
    @staticmethod
    def create_pipeline(source: str, **kwargs) -> ModelPipelineBase:
        """
        Factory method to instantiate a pipeline based on the provided source.
        :param source: A string indicating the source, e.g., "huggingface", "local", "api", or "gpt".
        :param kwargs: Additional parameters for pipeline initialization.
            For HuggingFaceDetailedPipeline:
              - model_name (str): Required.
              - cache_dir (str): Optional; if not provided and not in environment, no cache_dir is used.
              - torch_dtype (torch.dtype): Optional; default torch.float16.
              - temperature (float): Optional; default 0.0.
              - device (int): Optional; default -1 (CPU).
              - max_length (int): Optional; default 150.
            For GPTAPIPipeline:
              - model (str): Optional; default "gpt-3.5-turbo".
              - temperature (float): Optional; default 0.0.
              - max_tokens (int): Optional; default 150.
        :return: An instance of ModelPipelineBase.
        """
        source = source.lower()
        if source in {"huggingface", "local"}:
            if "model_name" not in kwargs:
                raise ValueError("model_name must be provided for HuggingFace pipeline.")
            return HuggingFaceDetailedPipeline(
                model_name=kwargs["model_name"],
                cache_dir=kwargs.get("cache_dir"),
                torch_dtype=kwargs.get("torch_dtype", torch.float16),
                temperature=kwargs.get("temperature", 0.0),
                device=kwargs.get("device", -1),
                max_length=kwargs.get("max_length", 150)
            )
        elif source in {"api", "gpt"}:
            return GPTAPIPipeline(
                model=kwargs.get("model", "gpt-3.5-turbo"),
                temperature=kwargs.get("temperature", 0.0),
                max_tokens=kwargs.get("max_tokens", 150)
            )
        else:
            raise ValueError(f"Unsupported source: {source}")

In [ ]:
def assemble_initial_prompt(transcript_dict: dict, with_context: bool, agent: str) -> str:
    """
    Assemble a prompt instructing the agent to generate its INITIAL answer.

    The prompt includes the question and, if requested, the context.
    It then uses the appropriate template from DEBATER, replacing placeholders
    for QUESTION, CONTEXT, AGENT, and leaves {ANSWER} as a marker.

    Parameters:
      transcript_dict (dict): Should contain at least "question" and optionally "context".
      with_context (bool): If True, use the template including context.
      agent (str): The agent identifier (e.g., "A" or "B").

    Returns:
      A formatted prompt string.
    """
    question = transcript_dict.get("question", "")

    if with_context:
        context = transcript_dict.get("context", "")
        template = DEBATER.get("initial_answer_prompt_with_context", "")
        prompt = template.format(QUESTION=question, CONTEXT=context)
    else:
        template = DEBATER.get("initial_answer_prompt_without_context", "")
        prompt = template.format(QUESTION=question)

    return prompt


def assemble_transcript(
    transcript_dict: dict,
    num_rounds: int,
    with_context: bool,
    agent: str,
    history_mode: str,   # "full_history" or "last_round_history"
    debate_type: str,    # "self" or "both"
    debate_mode: str,    # "asymmetric" or "symmetric"
    role: str = None     # For asymmetric mode: "challenger" or "defender"
) -> str:
    """
    Assemble a prompt for a debater.

    Parameters:
      transcript_dict: contains "question", "context", initial answers, and round responses.
      num_rounds:      number of rounds completed so far.
      with_context:    whether to include the context block.
      agent:           "A" or "B".
      history_mode:    "full_history" or "last_round_history".
      debate_type:     "self" (only own initial) or "both" (both agents' initials).
      debate_mode:     "asymmetric" or "symmetric".
      role:            for asymmetric: "challenger" or "defender"; ignored if symmetric.

    Returns:
      A fully formatted prompt string for the agent.
    """
    lines = []

    # 1) Pick the right instruction block based on symmetric/asymmetric + role
    if debate_mode.lower() == "asymmetric":
        if role is None:
            instruction = "Error: Asymmetric debate mode requires a role."
        elif role.lower() == "challenger":
            instruction = DEBATER["debater_ch_instru"]
        elif role.lower() == "defender":
            instruction = DEBATER["debater_de_instru"]
        else:
            instruction = "Error: Unknown role provided."
    elif debate_mode.lower() == "symmetric":
        instruction = DEBATER["debater_sym_instru"]
    else:
        instruction = "Error: Unknown debate mode provided."

    # 1.a) Prepend context‐reliability guidance
    context_trust = DEBATER.get("context_trust_instru", "")
    instruction = f"{context_trust}\n\n{instruction}"
    lines.append("Instruction:")
    lines.append(instruction)
    lines.append("────────────────────────────")

    # 2) Question and optional context
    question = transcript_dict.get("question", "")
    lines.append(f"Question: {question}")

    if with_context:
        context = transcript_dict.get("context", "")
        if context:
            lines.append(f"Context: {context}")
    lines.append("")

    # 3) If no rounds yet, ask for an initial answer
    if num_rounds == 0:
        lines.append(f"Agent {agent}, please provide your INITIAL ANSWER based on the above information.")
        return "\n".join(lines)

    # 4) Show initial answers
    lines.append("Initial Answer(s):")
    if debate_type.lower() == "self":
        key = f"debater_{agent}_initial"
        lines.append(f"  Agent {agent}: {transcript_dict.get(key,'')}")
    elif debate_type.lower() == "both":
        lines.append(f"  Agent A: {transcript_dict.get('debater_A_initial','')}")
        lines.append(f"  Agent B: {transcript_dict.get('debater_B_initial','')}")
    else:
        lines.append("Error: Unknown debate_type provided.")
    lines.append("")

    # 5) Show previous round(s)
    if history_mode.lower() == "full_history":
        lines.append("Previous Round Responses (Full History):")
        for r in range(1, num_rounds):
            lines.append(f"  Round {r}:")
            lines.append(f"    Agent A: {transcript_dict.get(f'debater_A_round_{r}','')}")
            lines.append(f"    Agent B: {transcript_dict.get(f'debater_B_round_{r}','')}")
    elif history_mode.lower() == "last_round_history":
        r = num_rounds
        lines.append("Previous Round Responses (Last Round):")
        lines.append(f"  Round {r}:")
        lines.append(f"    Agent A: {transcript_dict.get(f'debater_A_round_{r}','')}")
        lines.append(f"    Agent B: {transcript_dict.get(f'debater_B_round_{r}','')}")
    else:
        lines.append("Error: Unknown history_mode provided.")
    lines.append("")

    # 6) End‐of‐round thinking prompt
    if num_rounds == 1:
        end_instr = DEBATER["first_round_thinking"]
    elif num_rounds == 2:
        end_instr = DEBATER["second_round_thinking"]
    else:
        end_instr = DEBATER["nth_round_thinking"]

    lines.append("End Instruction:")
    lines.append(end_instr)
    lines.append("")
    lines.append(f"Agent {agent}, please provide your NEXT RESPONSE based on the above conversation.")

    return "\n".join(lines)

def assemble_judge_prompt(
    transcript_dict: dict,
    num_rounds: int,
    with_context: bool,
    judge_type: str = "symmetric"
) -> str:
    """
    Assemble a prompt for the judge agent based on the debate transcript dictionary.

    The prompt includes:
      - A judge-specific instruction from JUDGE (selected based on judge_type),
      - The complete debate transcript, including the question, (optionally) context, initial answers, and all round responses.
      - A final instruction to generate the final decision, which is provided by the JUDGE template.

    Parameters:
      transcript_dict (dict): Contains "question", "context", initial answers, and round responses.
      num_rounds (int): Number of completed debate rounds.
      with_context (bool): Whether to include context.
      judge_type (str): "symmetric" or "asymmetric" to select the appropriate judge instruction.

    Returns:
      A formatted prompt string for the judge.
    """
    lines = []

    # Select judge instruction.
    if judge_type.lower() == "symmetric":
        judge_instruction = JUDGE.get("judge_sym_instru", "")
    elif judge_type.lower() == "asymmetric":
        judge_instruction = JUDGE.get("judge_asym_instru", "")
    else:
        judge_instruction = "Error: Unknown judge type provided."

    lines.append("Judge Instruction:")
    lines.append(judge_instruction)
    lines.append("────────────────────────────")

    # Add the question.
    question = transcript_dict.get("question", "")
    lines.append("Question: {QUESTION}".format(QUESTION=question))

    # Optionally, add context.
    if with_context:
        context = transcript_dict.get("context", "")
        if context:
            lines.append("Context: {CONTEXT}".format(CONTEXT=context))
    lines.append("")

    # Add initial answers.
    lines.append("Initial Answers:")
    lines.append("  Agent A: " + transcript_dict.get("debater_A_initial", ""))
    lines.append("  Agent B: " + transcript_dict.get("debater_B_initial", ""))
    lines.append("")

    # Add round responses (full history is preferred for the judge).
    if num_rounds > 0:
        lines.append("Round Responses:")
        for r in range(1, num_rounds):
            lines.append("  Round {r}:".format(r=r))
            lines.append("    Agent A: " + transcript_dict.get(f"debater_A_round_{r}", ""))
            lines.append("    Agent B: " + transcript_dict.get(f"debater_B_round_{r}", ""))
    else:
        lines.append("No debate rounds have been completed yet.")
    lines.append("")

    # Use the final decision prompt from JUDGE.
    final_instr = JUDGE.get("final_decision_prompt", "")
    lines.append(final_instr)

    return "\n".join(lines)

In [ ]:
import re
from typing import Optional

class Debater:
    def __init__(
        self,
        name: str,
        pipeline: ModelPipelineBase,
        has_context: bool = False,
        context: Optional[str] = None,
        min_words: int = 5,
        max_words: int = 50,
        debate_mode: str = "symmetric",
        debate_type: str = "both",
        history_mode: str = "full_history",
        role: Optional[str] = None,
    ):
        self.name          = name
        self.pipeline      = pipeline
        self.has_context   = has_context
        self.context       = context
        self.min_words     = min_words
        self.max_words     = max_words
        self.debate_mode   = debate_mode       # "symmetric" or "asymmetric"
        self.debate_type = debate_type     # "self" or "both"
        self.history_mode  = history_mode      # "full_history" or "last_round_history"
        self.role          = role              # only used if asym

    def get_initial_answer(self, transcript_dict: dict) -> str:
        prompt  = assemble_initial_prompt(transcript_dict,
                                         with_context=self.has_context,
                                         agent=self.name)
        response = self.pipeline.call_model(prompt)
        # extract after "answer:"
        #m = re.search(r"answer:\s*(.*)", response, re.IGNORECASE)
        #return m.group(1).strip() if m else response
        return response

    def debate_response(self, transcript_dict: dict, round_num: int) -> str:
        prompt = assemble_transcript(
            transcript_dict,
            num_rounds=round_num,
            with_context=self.has_context,
            agent=self.name,
            history_mode=self.history_mode,
            debate_type=self.debate_type,   # “self” or “both”
            debate_mode=self.debate_mode,   # “symmetric” or “asymmetric”
            role=self.role,
        )
        return self.pipeline.call_model(prompt)

    def extract_argument(self, response: str) -> str:
        """
        Extract the text within <argument>...</argument> tags.
        """
        match = re.search(r"<argument>(.*?)</argument>", response)
        if match:
            return match.group(1).strip()
        return response.strip()

    def truncate(self, argument: str) -> str:
        """
        Truncate the argument to the maximum word count.
        """
        words = argument.split()
        if len(words) > self.max_words:
            truncated = " ".join(words[:self.max_words]) + "... <TRUNCATED>"
            return truncated
        return argument


class Judge:
    def __init__(self, name: str, pipeline: ModelPipelineBase):
        """
        Initialize the Judge.
        """
        self.name = name
        self.pipeline = pipeline

    def decide(self, transcript: str) -> str:
        """
        Generate a final decision based on the debate transcript.
        Uses the appropriate judge instruction from the JUDGE template.
        """
        prompt = JUDGE.get("judge_sym_instru", "").format(name=self.name, transcript=transcript)
        response = self.pipeline.call_model(prompt)
        decision = self.extract_clean_argument(response)
        return f"[{self.name}] Final Decision: {decision}"

    def extract_clean_argument(self, response: str) -> str:
        """
        Extract the text within <argument>...</argument> tags from the judge's response.
        """
        match = re.search(r"<argument>(.*?)</argument>", response)
        if match:
            return match.group(1).strip()
        return response.strip()

In [ ]:
def _make_jsonl_line(custom_id, prompt, max_tokens, temp=0.0):
    body = {"model": MODEL_ID,
            "messages":[{"role":"user","content":prompt}],
            "max_tokens":max_tokens,"temperature":temp}
    return json.dumps({"custom_id":custom_id,"method":"POST",
                       "url":"/v1/chat/completions","body":body},
                       ensure_ascii=False)

def _upload_and_batch(lines):
    fn = tempfile.mktemp(suffix=".jsonl")
    Path(fn).write_text("\n".join(lines),encoding="utf-8")
    file_id = client.files.create(file=open(fn,"rb"), purpose="batch").id
    batch   = client.batches.create(input_file_id=file_id,
                                    endpoint="/v1/chat/completions",
                                    completion_window="24h")
    while batch.status not in {"completed","failed","expired","cancelled"}:
        time.sleep(10)
        batch = client.batches.retrieve(batch.id)
    txt = client.files.content(batch.output_file_id).text if batch.output_file_id else ""
    out={}
    for ln in txt.strip().splitlines():
        rec=json.loads(ln)
        out[rec["custom_id"]] = (rec["response"]["body"]["choices"][0]
                                  ["message"]["content"]
                                  if rec["error"] is None else
                                  f"[ERROR:{rec['error']['code']}]")
    return out

# shortcut
def L(cid,prompt,max_tokens=250,temp=0.0): return _make_jsonl_line(cid,prompt,max_tokens,temp)


In [ ]:


YEAR_RE = re.compile(r"\b(\d{4})\b")
def year(txt): m=YEAR_RE.search(txt); return m.group(1) if m else txt.strip()


In [ ]:



def run_condition(order: str, total_rounds: int = 3) -> List[Dict[str, Any]]:
    """
    order ∈ {"A_first", "B_first", "simul"}
    total_rounds: how many debate rounds AFTER round-0
    Returns tasks with prompts/answers plus judge verdicts for EVERY round.
    """
    tasks = deepcopy(tasks_base)

    # Round-0 independent answers
    batch = []
    for i, t in enumerate(tasks):
        pA = INIT_CTX.format(QUESTION=t["question"], CONTEXT=t["context"])
        pB = INIT_NO .format(QUESTION=t["question"])
        t["prompt_A0"], t["prompt_B0"] = pA, pB
        batch.append(L(f"A0-{i}", pA, 10))
        batch.append(L(f"B0-{i}", pB, 10))
    res = _upload_and_batch(batch)
    for i, t in enumerate(tasks):
        t["A0"] = year(res[f"A0-{i}"])
        t["B0"] = year(res[f"B0-{i}"])

    # helper to craft prompts
    def make_prompt(agent: str, t: dict, r: int) -> str:
        opp = "B" if agent == "A" else "A"
        ctx = CTX_BLK.format(CONTEXT=t["context"]) if agent == "A" else ""
        base = INTRO.format(NAME=agent, QUESTION=t["question"],
                            CTXBLOCK=ctx, ANS=t[f"{agent}0"])

        if r == 1:                                     # opening debate
            first = (order == "A_first" and agent == "A") or \
                    (order == "B_first" and agent == "B") or \
                    (order == "simul")
            if first:
                return base + ROLE_ATTACK
            return base + f"Opponent’s answer: {t[f'{opp}1']}\n\n" + ROLE_ATTACK

        # r ≥ 2
        hist = "".join(
            f"Round {k}: A:{t[f'A{k}']} B:{t[f'B{k}']}\n" for k in range(1, r)
        )
        first = (order == "A_first" and agent == "A") or \
                (order == "B_first" and agent == "B")
        if order == "simul":
            opp_line = f"Opponent’s last answer: {t[f'{opp}{r-1}']}\n\n"
        elif first:
            opp_line = ""
        else:
            opp_line = f"Opponent’s current answer: {t[f'{opp}{r}']}\n\n"

        return base + ROUND_HEAD.format(R=r, HIST=hist) + opp_line + ROLE_DEFEND

    # loop over debate rounds
    for r in range(1, total_rounds + 1):
        # produce A_r and B_r
        if order == "simul":
            lines = []
            for i, t in enumerate(tasks):
                pA = make_prompt("A", t, r); t[f"prompt_A{r}"] = pA
                pB = make_prompt("B", t, r); t[f"prompt_B{r}"] = pB
                lines += [L(f"A{r}-{i}", pA), L(f"B{r}-{i}", pB)]
            res = _upload_and_batch(lines)
            for i, t in enumerate(tasks):
                t[f"A{r}"] = res[f"A{r}-{i}"]
                t[f"B{r}"] = res[f"B{r}-{i}"]

        elif order == "A_first":
            lines = [L(f"A{r}-{i}", make_prompt("A", t, r)) for i, t in enumerate(tasks)]
            res = _upload_and_batch(lines)
            for i, t in enumerate(tasks):
                t[f"A{r}"] = res[f"A{r}-{i}"]

            lines = [L(f"B{r}-{i}", make_prompt("B", t, r)) for i, t in enumerate(tasks)]
            res = _upload_and_batch(lines)
            for i, t in enumerate(tasks):
                t[f"B{r}"] = res[f"B{r}-{i}"]

        else:  # B_first
            lines = [L(f"B{r}-{i}", make_prompt("B", t, r)) for i, t in enumerate(tasks)]
            res = _upload_and_batch(lines)
            for i, t in enumerate(tasks):
                t[f"B{r}"] = res[f"B{r}-{i}"]

            lines = [L(f"A{r}-{i}", make_prompt("A", t, r)) for i, t in enumerate(tasks)]
            res = _upload_and_batch(lines)
            for i, t in enumerate(tasks):
                t[f"A{r}"] = res[f"A{r}-{i}"]

        print(f"{order}: finished round {r}")

        # judge after this round
        jlines = []
        for i, t in enumerate(tasks):
            transcript = (
                f"Round 0: A={t['A0']}  B={t['B0']}\n" +
                "".join(
                    f"Round {k}: A:{t[f'A{k}']}  B:{t[f'B{k}']}\n"
                    for k in range(1, r + 1)
                )
            )
            jprompt = (
                f"Question: {t['question']}\n"
                f"Transcript up to round {r}:\n{transcript}\n\n"
                f"{JUDGE_TAIL}"
            )
            t[f"judge_r{r}_prompt"] = jprompt
            jlines.append(L(f"J{r}-{i}", jprompt, TOK_JUDGE))
        jres = _upload_and_batch(jlines)
        for i, t in enumerate(tasks):
            raw = jres[f"J{r}-{i}"]
            t[f"judge_r{r}_output"] = raw
            t[f"judge_r{r}_year"] = year(raw)

    return tasks

In [ ]:

MODEL_ID   = ""
DATA_FILE  = Path("")
tasks_base = json.load(DATA_FILE.open())
#tasks_base[:10]
ROUNDS     = 3
ORDER_MODES= ["A_first","B_first","simul"]
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
TOK_JUDGE = 200
YEAR_RE    = re.compile(r"\b(\d{4})\b")

In [ ]:
# ---------- 6.  Run all three modes & save --------------------
name_list = []
for mode in ORDER_MODES:
    result=run_condition(mode)
    ts=datetime.now().strftime("%Y%m%d_%H%M%S")
    fname=f"judge_debate_{mode}_R{ROUNDS}_{MODEL_ID}_{ts}.json"
    name_list.append(fname)
    json.dump(result,(Path(".")/fname).open("w",encoding="utf-8"),indent=2)
    print("✅ saved", fname)

A_first : finished round 1
A_first : finished round 2
A_first : finished round 3
✅ saved judge_debate_A_first_R3_gpt-3.5-turbo_20250524_123908.json
B_first : finished round 1
B_first : finished round 2
B_first : finished round 3
✅ saved judge_debate_B_first_R3_gpt-3.5-turbo_20250524_152618.json
simul : finished round 1
simul : finished round 2
simul : finished round 3
✅ saved judge_debate_simul_R3_gpt-3.5-turbo_20250524_170353.json


In [ ]:
import json, re, os, time, tempfile
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any
from openai import OpenAI

number_rounds = 2

def year(txt:str)->str: m=YEAR_RE.search(txt); return m.group(1) if m else txt.strip()

FILES = name_list
FILES = [Path(f) for f in FILES]

# Batch helpers (same as before)
client = OpenAI()

def make_line(cid,prompt,max_tok=TOK_JUDGE,temp=0.0):
    body={"model":MODEL_ID,"messages":[{"role":"user","content":prompt}],
          "max_tokens":max_tok,"temperature":temp}
    return json.dumps({"custom_id":cid,"method":"POST",
                       "url":"/v1/chat/completions","body":body},
                      ensure_ascii=False)

def batch_run(lines):
    fn=tempfile.mktemp(suffix=".jsonl")
    Path(fn).write_text("\n".join(lines), encoding="utf-8")
    fid=client.files.create(file=open(fn,"rb"),purpose="batch").id
    bid=client.batches.create(input_file_id=fid,endpoint="/v1/chat/completions",
                              completion_window="24h").id
    while True:
        b=client.batches.retrieve(bid)
        if b.status in {"completed","failed","expired","cancelled"}:
            break
        time.sleep(8)
    txt=client.files.content(b.output_file_id).text if b.output_file_id else ""
    out={}
    for ln in txt.strip().splitlines():
        rec=json.loads(ln)
        out[rec["custom_id"]]=(rec["response"]["body"]["choices"][0]
                               ["message"]["content"]
                               if rec["error"] is None else
                               f"[ERROR:{rec['error']['code']}]")
    return out


for fpath in FILES:
    tasks=json.loads(fpath.read_text())

    for r in (1,number_rounds):
        # build all prompts for this round
        lines=[]
        for i,t in enumerate(tasks):
            transcript = (
                f"Round 0: A={t['A0']}  B={t['B0']}\n" +
                "".join(
                    f"Round {k}: A:{t[f'A{k}']}  B:{t[f'B{k}']}\n"
                    for k in range(1, r + 1)
                )
            )
            jp=(f"Question: {t['question']}\n"
                f"Debate transcript (up to round {r}):\n{transcript}\n\n"
                f"{JUDGE_TAIL}")
            t[f"judge_r{r}_prompt"]=jp      # store prompt
            lines.append(make_line(f"J{r}-{i}", jp))
        # batch run
        res=batch_run(lines)
        for i,t in enumerate(tasks):
            raw=res[f"J{r}-{i}"]
            t[f"judge_r{r}_output"]=raw
            t[f"judge_r{r}_year"]=year(raw)

    # save new file
    out_name=fpath.with_name(fpath.stem+"_J12.json")
    out_name.write_text(json.dumps(tasks,indent=2,ensure_ascii=False))
    print("✅ wrote", out_name)

✅ wrote judge_debate_A_first_R3_gpt-3.5-turbo_20250524_123908_J12.json
✅ wrote judge_debate_B_first_R3_gpt-3.5-turbo_20250524_152618_J12.json
✅ wrote judge_debate_simul_R3_gpt-3.5-turbo_20250524_170353_J12.json


#### R3 to R5

In [ ]:

#  Extend existing Rn debate files to full m rounds + judges

import json, re, os, time, tempfile
from pathlib import Path
from datetime import datetime
from copy import deepcopy
from openai import OpenAI

# config
MODEL_ID   = "gpt-3.5-turbo"
TOK_DEB    = 250
TOK_JUDGE  = 120
NEW_ROUNDS = []      # add these rounds
YEAR_RE    = re.compile(r"\b(\d{4})\b")
def year(x): m=YEAR_RE.search(str(x)); return m.group(1) if m else str(x).strip()


FILES = {

}
FILES = [Path(f) for f in FILES]


client = OpenAI()

def make_line(cid, prompt, max_tok, temp=0.0):
    body={"model":MODEL_ID,"messages":[{"role":"user","content":prompt}],
          "max_tokens":max_tok,"temperature":temp}
    return json.dumps({"custom_id":cid,"method":"POST",
                       "url":"/v1/chat/completions","body":body},
                      ensure_ascii=False)

def batch_run(lines):
    fn=tempfile.mktemp(suffix=".jsonl")
    Path(fn).write_text("\n".join(lines), encoding="utf-8")
    fid=client.files.create(file=open(fn,"rb"),purpose="batch").id
    bid=client.batches.create(input_file_id=fid,endpoint="/v1/chat/completions",
                              completion_window="24h").id
    while True:
        b=client.batches.retrieve(bid)
        if b.status in {"completed","failed","expired","cancelled"}:
            break
        time.sleep(8)
    txt=client.files.content(b.output_file_id).text if b.output_file_id else ""
    out={}
    for ln in txt.strip().splitlines():
        rec=json.loads(ln)
        out[rec["custom_id"]]=(rec["response"]["body"]["choices"][0]
                               ["message"]["content"]
                               if rec["error"] is None else
                               f"[ERROR:{rec['error']['code']}]")
    return out


for fpath in FILES:
    mode = ("A_first" if "A_first" in fpath.name else
            "B_first" if "B_first" in fpath.name else "simul")
    tasks = json.loads(fpath.read_text())

    def make_prompt(agent:str, t:dict, r:int):
        opp   = "B" if agent=="A" else "A"
        ctx   = CTX_BLK.format(CONTEXT=t["context"]) if agent=="A" else ""
        base  = INTRO.format(NAME=agent, QUESTION=t["question"],
                             CTXBLOCK=ctx, ANS=t[f"{agent}0"])
        hist  = "".join(f"Round {k}: A:{t[f'A{k}']}  B:{t[f'B{k}']}\n"
                        for k in range(1, r))
        first = (mode=="A_first" and agent=="A") or \
                (mode=="B_first" and agent=="B")
        if mode=="simul":
            opp_line = f"Opponent’s last answer: {t[f'{opp}{r-1}']}\n\n"
        elif first:
            opp_line = ""     # first speaker doesn't see current-round opp
        else:
            opp_line = f"Opponent’s current answer: {t[f'{opp}{r}']}\n\n"
        return base + ROUND_HEAD.format(R=r, HIST=hist) + opp_line + ROLE_DEFEND

    for r in NEW_ROUNDS:
        if mode=="simul":
            batch=[]
            for i,t in enumerate(tasks):
                pA=make_prompt("A",t,r); t[f"prompt_A{r}"]=pA
                pB=make_prompt("B",t,r); t[f"prompt_B{r}"]=pB
                batch.append(make_line(f"A{r}-{i}", pA, TOK_DEB))
                batch.append(make_line(f"B{r}-{i}", pB, TOK_DEB))
            res=batch_run(batch)
            for i,t in enumerate(tasks):
                t[f"A{r}"]=res[f"A{r}-{i}"]; t[f"B{r}"]=res[f"B{r}-{i}"]

        elif mode=="A_first":
            # A speaks
            batch=[make_line(f"A{r}-{i}", make_prompt("A",t,r), TOK_DEB)
                   for i,t in enumerate(tasks)]
            res=batch_run(batch)
            for i,t in enumerate(tasks): t[f"A{r}"]=res[f"A{r}-{i}"]
            # B replies
            batch=[make_line(f"B{r}-{i}", make_prompt("B",t,r), TOK_DEB)
                   for i,t in enumerate(tasks)]
            res=batch_run(batch)
            for i,t in enumerate(tasks): t[f"B{r}"]=res[f"B{r}-{i}"]

        else:  # B_first
            batch=[make_line(f"B{r}-{i}", make_prompt("B",t,r), TOK_DEB)
                   for i,t in enumerate(tasks)]
            res=batch_run(batch)
            for i,t in enumerate(tasks): t[f"B{r}"]=res[f"B{r}-{i}"]
            batch=[make_line(f"A{r}-{i}", make_prompt("A",t,r), TOK_DEB)
                   for i,t in enumerate(tasks)]
            res=batch_run(batch)
            for i,t in enumerate(tasks): t[f"A{r}"]=res[f"A{r}-{i}"]

        print(mode,": finished new round", r)


        jlines=[]
        for i,t in enumerate(tasks):
            transcript="Round 0: A="+t["A0"]+"  B="+t["B0"]+"\n"+ \
                       "".join(f"Round {k}: A:{t[f'A{k}']}  B:{t[f'B{k}']}\n"
                               for k in range(1,r+1))
            jp=(f"Question: {t['question']}\n"
                f"Transcript up to round {r}:\n{transcript}\n\n"
                f"{JUDGE_TAIL}")
            t[f"judge_r{r}_prompt"]=jp
            jlines.append(make_line(f"J{r}-{i}", jp, TOK_JUDGE))
        res=batch_run(jlines)
        for i,t in enumerate(tasks):
            raw=res[f"J{r}-{i}"]
            t[f"judge_r{r}_output"]=raw
            t[f"judge_r{r}_year"]=year(raw)

    out_name = fpath.with_name(fpath.stem+"_extend.json")
    out_name.write_text(json.dumps(tasks, indent=2, ensure_ascii=False))
    print("✅ wrote", out_name)

simul : finished new round 4
simul : finished new round 5
✅ wrote judge_debate_simul_R5_gpt-3.5-turbo_20250520_200713_J12_J12345.json
A_first : finished new round 4
A_first : finished new round 5
✅ wrote judge_debate_A_first_R5_gpt-3.5-turbo_20250520_134000_J12_J12345.json
B_first : finished new round 4
B_first : finished new round 5
✅ wrote judge_debate_B_first_R5_gpt-3.5-turbo_20250520_162208_J12_J12345.json
